In [1]:

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pickle



In [2]:

# Training Parameters
learning_rate = 0.0001
training_steps = 500
display_step = 100
batch_size = 50

# Network Parameters
num_input = 67*27 # 67 x 27 is size of each frame
max_timesteps = 128
timesteps = 128    # to be assigned for each training data separately
num_hidden = 64  # hidden layer num of features
num_classes = 10  # 10 actions

# tf Graph input
X = tf.placeholder("float", [None, max_timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}



def next_batch(x,labels,timesteps,batch_size):
    n = len(x)
    ind = np.random.randint(0,n,batch_size)
    batch_x = np.zeros(shape=(batch_size, max_timesteps, 67*27), dtype=np.float32)
    temp = [np.reshape(np.transpose(x[i][...,0:max_timesteps],(2,0,1)),(-1,67*27)) for i in ind]
    for i in range(batch_size):
        batch_x[i,:temp[i].shape[0],:] = temp[i]
    batch_y = np.eye(num_classes)[labels[ind].astype(np.uint8)]
    return batch_x, batch_y



def RNN(x, weights, biases):

    # x = tf.unstack(x, axis=1, num=max_timesteps)
    # here is a space to define your LSTM machine
    lstm_cell = tf.nn.rnn_cell.LSTMCell(num_hidden)
    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                   inputs=x,
                                   dtype=tf.float32)
    return tf.matmul(outputs[:,-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

x = pickle.load(open('X.pickle', 'rb'), encoding='latin1')


labels = np.load('l.npy')


# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = next_batch(x,labels,timesteps,batch_size)
        # define the optimization procedure
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    # we would like to test our network - but there is not enough data... so we just learn training data by heart! <3
    


Step 1, Minibatch Loss= 2.8461, Training Accuracy= 0.120
Step 50, Minibatch Loss= 2.6581, Training Accuracy= 0.240
Step 100, Minibatch Loss= 2.6794, Training Accuracy= 0.220
Step 150, Minibatch Loss= 2.0339, Training Accuracy= 0.180
Step 200, Minibatch Loss= 1.8704, Training Accuracy= 0.200
Step 250, Minibatch Loss= 1.6973, Training Accuracy= 0.340
Step 300, Minibatch Loss= 1.5608, Training Accuracy= 0.280
Step 350, Minibatch Loss= 1.2671, Training Accuracy= 0.500
Step 400, Minibatch Loss= 1.4116, Training Accuracy= 0.500
Step 450, Minibatch Loss= 1.2249, Training Accuracy= 0.560
Step 500, Minibatch Loss= 1.3392, Training Accuracy= 0.460
Optimization Finished!
3.65812 0.0


Cross-validation is useless with this amount of data - network will suck hard on the validation data, no matter how many hidden layers...